In [1]:
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df_symptoms = pd.read_csv('test_data/results_symptoms.csv')

In [4]:
len(df_symptoms)

93995

In [5]:
df_symptoms.columns

Index(['R', 'temperature', 'high_risk_exposure_occupation',
       'high_risk_interactions', 'labored_respiration', 'rhonchi', 'cough',
       'fever', 'sob', 'diarrhea', 'fatigue', 'headache', 'loss_of_smell',
       'loss_of_taste', 'runny_nose', 'muscle_sore', 'sore_throat', 'wheezes'],
      dtype='object')

In [6]:
#df_symptoms = df_symptoms[df_symptoms > 0.5].fillna(0)

In [7]:
#df_symptoms.to_csv('test_data/results_symptoms.csv')

In [8]:
df_symptoms

,R,temperature,high_risk_exposure_occupation,high_risk_interactions,labored_respiration,rhonchi,cough,fever,sob,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat,wheezes
0,0.0,36.950000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,36.750000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,36.950000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,36.750000,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,36.250000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93990,0.0,36.794765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93991,0.0,36.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93992,0.0,36.794765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93993,0.0,36.794765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df_symptoms.describe()

,R,temperature,high_risk_exposure_occupation,high_risk_interactions,labored_respiration,rhonchi,cough,fever,sob,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat,wheezes
count,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000,93995.000000
mean,0.013969,36.794765,0.073770,0.151231,0.004128,0.075398,0.062184,0.021533,0.030236,0.020182,0.065025,0.057673,0.007054,0.007192,0.036800,0.036066,0.062854,0.101442
std,0.117362,0.205696,0.261397,0.358276,0.064116,0.264033,0.241491,0.145154,0.171236,0.140623,0.246571,0.233126,0.083689,0.084500,0.188271,0.186455,0.242702,0.301914
min,0.000000,33.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,36.794765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,36.794765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,36.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,39.600000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support as score

In [11]:
#df_symptoms.dropna(inplace=True) # leaves 1946

In [ ]:
# normalize the temperaature
#df_symptoms['temperature'] = (df_symptoms['temperature'] - df_symptoms['temperature'].min()) / (df_symptoms['temperature'].max() - df_symptoms['temperature'].min())

In [12]:
df_symptoms.head(5)

,R,temperature,high_risk_exposure_occupation,high_risk_interactions,labored_respiration,rhonchi,cough,fever,sob,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat,wheezes
0,0.0,36.95,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,36.75,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,36.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,36.75,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,36.25,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#df_symptoms[df_symptoms['R'] == 1]
df_symptoms["id"] = df_symptoms.index

In [14]:
df_symptoms.iloc[:,2:-1].head()

,high_risk_exposure_occupation,high_risk_interactions,labored_respiration,rhonchi,cough,fever,sob,diarrhea,fatigue,headache,loss_of_smell,loss_of_taste,runny_nose,muscle_sore,sore_throat,wheezes
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
len(df_symptoms)

93995

In [16]:
#df_symptoms.head()

In [17]:
symptom_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier())
    ])

In [18]:
# don't include the ID as a feature
symptom_clf.fit(df_symptoms.iloc[:,2:-1], df_symptoms['R'])

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', RandomForestClassifier())])

In [19]:
symptom_clf['scaler'].scale_

array([0.26139603, 0.35827433, 0.06411583, 0.26403186, 0.24148973,
       0.14515298, 0.17123509, 0.14062224, 0.2465695 , 0.23312459,
       0.08368879, 0.0844994 , 0.18827002, 0.18645377, 0.24270091,
       0.30191253])

In [20]:
symptom_clf['clf'].feature_importances_

array([0.06596132, 0.07415236, 0.01602243, 0.02894927, 0.0720782 ,
       0.0795085 , 0.06688181, 0.06029997, 0.06461113, 0.06390115,
       0.10481658, 0.07243817, 0.06455038, 0.06248929, 0.06874587,
       0.03459357])

In [21]:
feature_df = pd.DataFrame({'feature':df_symptoms.columns[2:-1], 
                           'importance': symptom_clf['clf'].feature_importances_})

In [22]:
pysqldf("SELECT * FROM feature_df ORDER BY importance DESC")

,feature,importance
0,loss_of_smell,0.104817
1,fever,0.079508
2,high_risk_interactions,0.074152
3,loss_of_taste,0.072438
4,cough,0.072078
5,sore_throat,0.068746
6,sob,0.066882
7,high_risk_exposure_occupation,0.065961
8,fatigue,0.064611
9,runny_nose,0.064550


In [23]:
symptom_scores = cross_val_score(symptom_clf, df_symptoms.iloc[:,2:], df_symptoms['R'], cv=5)

In [24]:
print("Accuracy: %0.2f, StDev: %0.2f)" % (symptom_scores.mean(), symptom_scores.std()))

Accuracy: 0.28, StDev: 0.36)


In [25]:
test_predicted_proba = symptom_clf.predict_proba(df_symptoms.iloc[:, 2:-1])

In [26]:
test_predicted = symptom_clf.predict(df_symptoms.iloc[:, 2:-1])

In [27]:
df_pred = pd.DataFrame({"p_cat":test_predicted, "p_prob":[item[1] for item in test_predicted_proba], "p_id":df_symptoms.id })

In [31]:
pysqldf("SELECT * FROM df_pred ORDER BY p_prob DESC LIMIT 5")

,p_cat,p_prob,p_id
0,1.0,0.960000,982
1,1.0,0.960000,12258
2,1.0,0.960000,78650
3,1.0,0.953333,4271
4,1.0,0.953333,48515


In [32]:
symptom_clf.predict_proba([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

array([[0.99269161, 0.00730839]])

In [33]:
symptom_clf.predict_proba([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])

array([[0.574, 0.426]])

In [34]:
symptom_clf.predict_proba([list(df_symptoms.iloc[123][2:-1])])

array([[0.96750139, 0.03249861]])

In [35]:
symptom_clf.predict_proba([[1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

array([[0.98441399, 0.01558601]])

In [36]:
y_pred = symptom_clf.predict(df_symptoms.iloc[:, 2:-1])

In [37]:
len(y_pred), y_pred.sum()

(93995, 204.0)

### Accuracy: 

How often did the predicted value match the actual value

### Precision: 

When you predicted a positive result, how often was the actual value positive?

### Recall: 

When the actual value was positive, how often did you predict a positive value?

### Visual:

https://en.wikipedia.org/wiki/Precision_and_recall

In [44]:
# predict categories
y_pred = symptom_clf.predict(df_symptoms.iloc[:, 2:-1])

In [50]:
df_pred = pd.DataFrame({'pred': y_pred, 'actual': df_symptoms['R']})

In [54]:
pd.set_option('display.max_rows', None)

In [62]:
df_pred.sort_values(['actual'], ascending=[0]).head(10)

,pred,actual
17978,0.0,1.0
29463,0.0,1.0
11545,0.0,1.0
1858,0.0,1.0
16321,0.0,1.0
49235,0.0,1.0
25042,0.0,1.0
1006,1.0,1.0
1005,1.0,1.0
21295,0.0,1.0


In [38]:
precision, recall, fscore, train_support = score(df_symptoms['R'], y_pred, pos_label=1, average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==df_symptoms['R']).sum()/len(y_pred), 3)))

Precision: 0.926 / Recall: 0.144 / Accuracy: 0.988


In [ ]:
# could we find a better trade-off using probabilities?

In [63]:
y_pred_proba = symptom_clf.predict_proba(df_symptoms.iloc[:, 2:-1])

In [65]:
# find a few with positive scores
df_pred = pd.DataFrame({'pred': [i[1] for i in y_pred_proba], 'actual': df_symptoms['R']})

In [71]:
# look at the current threshold of .50, should we move it?
df_pred.sort_values(['pred'], ascending=[0])[200:220]

,pred,actual
74185,0.500897,0.0
20695,0.500897,1.0
8891,0.500897,0.0
13089,0.500897,1.0
52154,0.494238,0.0
49423,0.494238,1.0
59611,0.493290,0.0
58104,0.493290,1.0
26768,0.493290,0.0
62390,0.493290,1.0
